In [58]:
import cv2
import dlib
import os
import sys
import numpy as np
from PIL import Image
import imutils

In [127]:
def put_mask(face_image,mask_image,landmark):
    nose_bridge=np.zeros((4,2),dtype="int")
    for i in range(27,31):
        nose_bridge[i-27]=(landmark.part(i).x,landmark.part(i).y)
    nose_point=nose_bridge[len(nose_bridge)*1//4]
    nose_vector=np.array(nose_point)

    chin=np.zeros((17,2),dtype="int")
    for i in range(0,17):
        chin[i]=(landmark.part(i).x,landmark.part(i).y)
    chin_left_point=chin[len(chin)*1//8]
    chin_right_point=chin[len(chin)*7//8]
    chin_bottom_point=chin[len(chin)*1//2]
    chin_bottom_vector=np.array(chin_bottom_point)

    width=mask_image.width
    height=mask_image.height
    width_ratio=1.2
    new_height=int(np.linalg.norm(nose_vector-chin_bottom_vector))

    mask_left_image=mask_image.crop((0,0,width//2,height))
    mask_right_image=mask_image.crop((width//2,0,width,height))
    mask_left_image_width=get_distance_from_point(chin_left_point,chin_bottom_vector,nose_vector)
    mask_right_image_width=get_distance_from_point(chin_right_point,chin_bottom_vector,nose_vector)
    mask_left_image_width=int(mask_left_image_width*width_ratio)
    mask_right_image_width=int(mask_right_image_width*width_ratio)
    mask_left_image=mask_left_image.resize((mask_left_image_width,new_height))
    mask_right_image=mask_right_image.resize((mask_right_image_width,new_height))

    new_size=(mask_left_image_width+mask_right_image_width,new_height)
    masked_image=Image.new("RGBA",new_size)
    masked_image.paste(mask_left_image,(0,0),mask_left_image)
    masked_image.paste(mask_right_image,(mask_left_image.width,0),mask_right_image)

    angle=np.arctan2(chin_bottom_point[1]-nose_point[1],chin_bottom_point[0]-nose_point[0])
    rotated_mask_image=masked_image.rotate(angle,expand=True)

    center_X=(nose_point[0]+chin_bottom_point[0])//2
    center_Y=(nose_point[1]+chin_bottom_point[1])//2

    offset=masked_image.width//2-mask_left_image.width
    radian=angle*np.pi/180
    box_x=center_X+int(offset*np.cos(radian))-rotated_mask_image.width//2
    box_Y=center_Y+int(offset*np.sin(radian))-rotated_mask_image.height//2

    face_image.paste(masked_image,(box_x,box_Y),masked_image)    

    return face_image


In [128]:
def get_distance_from_point(point, line_point1, line_point2):
    distance = np.abs((line_point2[1] - line_point1[1]) * point[0] +
                      (line_point1[0] - line_point2[0]) * point[1] +
                      (line_point2[0] - line_point1[0]) * line_point1[1] +
                      (line_point1[1] - line_point2[1]) * line_point1[0]) / \
               np.sqrt((line_point2[1] - line_point1[1]) * (line_point2[1] - line_point1[1]) +
                       (line_point1[0] - line_point2[0]) * (line_point1[0] - line_point2[0]))
    return int(distance)

In [139]:
def pipeline(image_path="image_1.png",mask_path="mask_2.png",landmark_dat_file_path="shape_predictor_68_face_landmarks.dat",caffe_prototxt_path="deploy.prototxt.txt",caffe_model_path="res10_300x300_ssd_iter_140000.caffemodel",threshold=0.9):
    predictor=dlib.shape_predictor(landmark_dat_file_path)
    net=cv2.dnn.readNetFromCaffe(caffe_prototxt_path,caffe_model_path)
    frame=cv2.imread(image_path)
    tmp=frame.copy()
    cv2.imshow("frame",tmp)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    (h,w)=frame.shape[:2]
    blob=cv2.dnn.blobFromImage(cv2.resize(frame,(300,300)),1.0,(300,300),(104.0,177.0,123.0))
    net.setInput(blob)
    detections=net.forward()
    frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGBA)
    face_image=Image.fromarray(frame)
    mask_image=Image.open(mask_path).convert("RGBA")
    for i in range(detections.shape[2]):
        confidence=detections[0,0,i,2]
        if(confidence>threshold):
            box=detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startX,startY,endX,endY)=box.astype("int")
            rect=dlib.rectangle(startX,startY,endX,endY)
            landmark=predictor(tmp,rect)
            cv2.rectangle(tmp,(startX,startY),(endX,endY),(0,255,180),2)
            for i in range(68):
                cv2.circle(tmp,(landmark.part(i).x,landmark.part(i).y),1,(0,255,180),-1)
            face_image=put_mask(face_image,mask_image,landmark)
            face_image.save("masked_image_1.png")
    new_masked_face_image=np.array(face_image.convert("RGB"))
    cv2.imshow("frame",new_masked_face_image[:,:,::-1].copy())
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [140]:
pipeline()

In [141]:
def pipeline_heavy(image_paths,mask_paths,landmark_dat_file_path="shape_predictor_68_face_landmarks.dat",caffe_prototxt_path="deploy.prototxt.txt",caffe_model_path="res10_300x300_ssd_iter_140000.caffemodel",threshold=0.9):
    predictor=dlib.shape_predictor(landmark_dat_file_path)
    net=cv2.dnn.readNetFromCaffe(caffe_prototxt_path,caffe_model_path)
    i=0
    for image_path in image_paths:
        count+=1
        frame=cv2.imread(image_path)
        tmp=frame.copy()
#         cv2.imshow("frame",tmp)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
        (h,w)=frame.shape[:2]
        blob=cv2.dnn.blobFromImage(cv2.resize(frame,(300,300)),1.0,(300,300),(104.0,177.0,123.0))
        net.setInput(blob)
        detections=net.forward()
        frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGBA)
        face_image=Image.fromarray(frame)
#         mask_image=Image.open(mask_path).convert("RGBA")
        for i in range(detections.shape[2]):
            confidence=detections[0,0,i,2]
            if(confidence>threshold):
                box=detections[0,0,i,3:7]*np.array([w,h,w,h])
                (startX,startY,endX,endY)=box.astype("int")
                rect=dlib.rectangle(startX,startY,endX,endY)
                landmark=predictor(tmp,rect)
                cv2.rectangle(tmp,(startX,startY),(endX,endY),(0,255,180),2)
                for i in range(68):
                    cv2.circle(tmp,(landmark.part(i).x,landmark.part(i).y),1,(0,255,180),-1)
                for mask_path in mask_paths:
                    mask_image=Image.open(mask_path).convert("RGBA")
                    mask_color=mask_path.split('.')[-2]
                    face_image=put_mask(face_image,mask_image,landmark)
                    face_image.save(os.path.join("{mask_color}".format(mask_color=mask_color),"masked_image_{x}.png".format(x=count)))
        new_masked_face_image=np.array(face_image.convert("RGB"))
        cv2.imshow("frame",new_masked_face_image[:,:,::-1].copy())
        cv2.waitKey(0)
        cv2.destroyAllWindows()